In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import requests
import numpy as np
import os
from natsort import natsorted
import imageio
model = hub.load("https://tfhub.dev/google/film/1")

OSError: SavedModel file does not exist at: C:\Users\saura\AppData\Local\Temp\tfhub_modules\2bc8869ae42f586158bb5df0a65ce2103bcc940c\{saved_model.pbtxt|saved_model.pb}

In [2]:
_UINT8_MAX_F = float(np.iinfo(np.uint8).max)
def load_image(img_url: str):
    """Returns an image with shape [height, width, num_channels], with pixels in [0..1] range, and type np.float32."""
    image = imageio.imread(img_url)
    image = image.astype(np.float32) / _UINT8_MAX_F
    # image = np.expand_dims(image,2)
    # image = np.dstack([image,image,image])
    return image

In [3]:
# ALL INPUTS HERE
skips = [1]
# Specify the folder path containing the images
pth=r"\\10.99.68.178\Saurabh\manuscript_figs\data\EM_seg\authentic - Copy"

In [4]:
# image_files = [_ for _ in os.listdir(pth) if _.endswith(('tif','png','jpg'))]
# image_files = natsorted(image_files)
#
# image1 = load_image(os.path.join(pth,image_files[0])) #4D image

In [5]:
# Get a list of all image file names in the folder
image_files = [_ for _ in os.listdir(pth) if _.endswith(('tif','png','jpg'))]
image_files = natsorted(image_files)
# image_files

for skip_num in skips:
    # Specify the output folder for the generated middle images
    output_folder = os.path.join(pth,f'skip_{skip_num}')
    os.mkdir(output_folder)
    os.listdir(pth)

    times = np.linspace(0,1,skip_num+2)
    times = times[1:-1]

    loaded_images = image_files[::skip_num + 1]
    impairs = zip(loaded_images[:-1], loaded_images[1:])
    imtargets = [image_files[i+1:i + skip_num+1] for i in range(0, len(image_files), skip_num + 1)]

    for impair,imtarget in zip(impairs,imtargets):
        image1 = load_image(os.path.join(pth,impair[0]))
        image2 = load_image(os.path.join(pth,impair[1]))
        for idx,time in enumerate(times):
            time1 = np.array([time], dtype=np.float32)
            # Prepare the input for the interpolation
            input_data = {
                'time': np.array([time1], dtype=np.float32),
                'x0': np.expand_dims(image1, axis=0),
                'x1': np.expand_dims(image2, axis=0)
            }
              # Generate the interpolated mid-frame using the model
            mid_frame = model(input_data)
            generated_image = mid_frame['image'][0].numpy()
            # Convert the image to the appropriate range and type
            image_in_uint8_range = np.clip(generated_image * _UINT8_MAX_F, 0.0, _UINT8_MAX_F)
            image_in_uint8 = (image_in_uint8_range + 0.5).astype(np.uint8)
            # Generate the filename for the generated middle image (using the next image filename)
            filename = os.path.splitext(imtarget[idx])[0] + '.tif'
            #filename = imtarget[idx]
            # Save the generated middle image in the output folder
            output_path = os.path.join(output_folder, filename)
            imageio.imwrite(output_path, image_in_uint8, format='tif')


C:\Users\saura\AppData\Local\Temp\ipykernel_1240\4254188497.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_url)


In [ ]:
from PIL import Image
import os

# Path to the folder containing the images
folder_path = r"\\10.99.68.178\Saurabh\visium_trial\Xenium_brain\test_3_26_24"

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".png"):
        # Open the image
        image_path = os.path.join(folder_path, filename)
        img = Image.open(image_path)

        # Convert image to RGB mode (remove alpha channel)
        img = img.convert("RGB")

        # Save the image (overwrite the original)
        img.save(image_path)

print("Alpha channels removed from all PNG images in the folder.")